In [ ]:
from my_functions import central_wavelength, nb_fwhm, z_NB
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
import numpy as np
import pandas as pd

In [ ]:
selection = pd.read_csv('csv/selection.csv')
selection.head()

In [ ]:
from load_jpas_catalogs import load_minijpas_jnep

pm_flx, pm_err, x_im, y_im, tile_id, number, mjj_starprob, spCl,\
        photoz, photoz_chi_best, photoz_odds, RA, DEC = load_minijpas_jnep(selection=True)

In [ ]:
z_Arr = z_NB(selection['nb_sel'])
src_Arr = selection['src']
L_lya = selection['L_lya']
zspec = selection['SDSS_zspec']
odds = photoz_odds[src_Arr]
photo_z_sorted = photoz[src_Arr]
chi_best = photoz_chi_best[src_Arr]
starprob = mjj_starprob[src_Arr]

In [ ]:
xmsdss = pd.read_csv('csv/xmatch_sdss_dr12.csv', sep=',', header=1)
zph = xmsdss['zph'].to_numpy()
zph.shape

In [ ]:
from my_functions import load_filter_tags
import os.path as op
from astropy.table import Table
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

# Load HETDEX
path_to_cat = '/home/alberto/almacen/HETDEX_catalogs/hetdex_source_catalog_1'
path_to_agn = '/home/alberto/almacen/HETDEX_catalogs/agn_catalog_v1.0'
version = 'v3.2'

source_table = Table.read(op.join(path_to_cat, f'hetdex_sc1_{version}.ecsv'))
det_table = Table.read(op.join(path_to_cat, 'hetdex_sc1_detinfo_{}.ecsv'.format(version)))
xm_hetdex_id = np.load('npy/hetdex_crossmatch_ids.npy')

fname = f'{path_to_agn}/hetdex_agn.fits'
agn = Table.read(fname, format='fits', hdu=1)

N_sources = pm_flx.shape[1]
z_hetdex = np.ones(N_sources) * -1
z_hetdex_conf = np.ones(N_sources) * -1
L_lya_hetdex = np.ones(N_sources) * -1
EW_lya_hetdex = np.ones(N_sources) * -9999999999999
EW_lya_hetdex_err = np.ones(N_sources) * 9999
type_hetdex = np.zeros(N_sources).astype(str)
for src in range(N_sources):
    if xm_hetdex_id[src] > 0:
        wh = np.where(xm_hetdex_id[src] == source_table['source_id'])[0][0]
        wh_det = np.where(xm_hetdex_id[src] == det_table['source_id'])[0][0]
        wh_agn = np.where(
            (source_table['RA'][wh] == agn['ra'])
            & (source_table['DEC'][wh] == agn['dec'])
        )[0]

        z_hetdex[src] = source_table['z_hetdex'][wh]
        z_hetdex_conf[src] = source_table['z_hetdex_conf'][wh]
        type_hetdex[src] = source_table['source_type'][wh]
        if len(wh_agn) > 0:
            wh_agn = wh_agn[0]
            F_lya = agn['flux_LyA'][wh_agn] * 1e-17
            dL = cosmo.luminosity_distance(z_hetdex[src]).to(u.cm).value
            L_lya_hetdex[src] = np.log10(F_lya * 4*np.pi * dL ** 2)
            print(L_lya_hetdex[src])
        else:
            L_lya_hetdex[src] = np.log10(source_table['lum_lya'][wh])
            if type_hetdex[src] == 'lae':
                EW_lya_hetdex[src] = det_table['flux'][wh_det] / det_table['continuum'][wh_det]
                EW_lya_hetdex_err[src] = (
                    (det_table['flux_err'][wh_det] / det_table['continuum'][wh_det]) ** 2
                    + (det_table['flux'][wh_det] * det_table['continuum'][wh_det]**-2
                    * det_table['continuum_err'][wh_det]) ** 2
                ) ** 0.5

def redshift(nb):
    return f'{z_NB(nb)[0]:0.2f}'
filter_tags = load_filter_tags()
def filter_name(nb):
    return filter_tags[nb]

src_list = src_Arr
z_hetdex_sorted = np.ones(src_list.shape) * -1
for i, src in enumerate(src_list):
    z_hetdex_sorted[i] = z_hetdex[np.arange(N_sources) == src]

nice_z_hetdex = np.abs(z_Arr - z_hetdex_sorted) < 0.16

In [ ]:
sdss_mask = np.array(selection['SDSS_spCl'] != '-') 
puri_mask = np.array(selection['puri'] > -99)
L_mask = np.array(L_lya > 40)
cont_mask = sdss_mask & L_mask & puri_mask & (np.abs(zspec - z_Arr) > 0.3) & np.isfinite(zspec)
conf_mask = sdss_mask & L_mask & puri_mask & ~(np.abs(zspec - z_Arr) > 0.3) & np.isfinite(zspec)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 2.5))

general_mask = (z_Arr > 0) & (odds > 0.0) & (chi_best < 10000)
this_mask = general_mask & ~(cont_mask | conf_mask) & (starprob > 0.1)
ax.plot(photo_z_sorted[this_mask], z_Arr[this_mask],
        marker='x', ls='', c='gray', mfc='gray',
        markeredgewidth=2)
this_mask = general_mask & ~(cont_mask | conf_mask) & (starprob <= 0.1)
ax.plot(photo_z_sorted[this_mask], z_Arr[this_mask],
        marker='s', ls='', c='royalblue', mfc='none',
        markeredgewidth=2)
this_mask = general_mask & (cont_mask | ((z_hetdex_sorted > 0) & ~nice_z_hetdex))
ax.plot(photo_z_sorted[this_mask], z_Arr[this_mask],
        marker='o', ls='', c='r', mfc='none',
        markeredgewidth=2)
this_mask = general_mask & (conf_mask | nice_z_hetdex)
ax.plot(photo_z_sorted[this_mask], z_Arr[this_mask],
        marker='o', ls='', c='g', mfc='g',
        markeredgewidth=2)

# lines
zsp_xx = np.linspace(0, 5, 50)
w_lya = 1215.67
w_OII = 3727.10
w_OIII = (5008.24 + 4960.30) * 0.5
w_Hbeta = 4862.68
w_Halpha = 6564.61
w_MgII = 2799.12
lines_dict = {
        'Lya': w_lya,
        'OII': w_OII,
        'OIII': w_OIII,
        'Hbeta': w_Hbeta,
        'Halpha': w_Halpha
}
ax.plot(zsp_xx, zsp_xx, ls='--', c='orange', label='Ly$\alpha$', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_OII - 1, zsp_xx, 
        ls='--', c='k', label='OII', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_OIII - 1, zsp_xx, 
        ls='--', c='k', label='OIII', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_Hbeta - 1, zsp_xx, 
        ls='--', c='k', label='Hbeta', zorder=-99)
# ax.plot((zsp_xx + 1) * w_lya / w_Halpha - 1, zsp_xx, 
#         ls='--', c='k', label='Halpha', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_MgII - 1, zsp_xx, 
        ls='--', c='k', label='MgII', zorder=-99)

text_plot = [[0.65, 3.9, 'OII'],
             [0.07, 3.9, 'OIII'],
             [0.28, 3.9, r'H$\beta$'],
             [1.2, 3.9, 'MgII']]
for text in text_plot:
        ax.text(text[0], text[1], text[2], fontsize=11)

# ax.set_yscale('log')
ax.set_ylim(2, 4.1)
ax.set_xlim(-0.1, 1.6)
ax.set_xlabel('$z_\mathrm{phot}$', fontsize=16)
ax.set_ylabel('$z_\mathrm{NB}$', fontsize=16)
ax.tick_params(direction='in', which='both', labelsize=14)
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

fig.tight_layout()
fig.savefig('figures/zphot_vs_z_Arr.pdf',
        bbox_inches='tight', facecolor='w', pad_inches=0.1)
plt.show()
